# Using a SurrealDB Knowedge Graph and Gemini to build a Graph-RAG 

This Jupyter Notebook illustrates the implementation and utilization of a graph-based Retrieval-Augmented Generation (RAG) system using a knowledge graph stored in SurrealDB. It focuses on leveraging the knowledge graph created by the graph_extractor.ipynb notebook for question answering tasks.

### Key Functionality
####  Data Loading:
        Loads the raw text data from which will be used for context in the prompt.

#### Leveraging the Existing Knowledge Graph:

        Create a SurrQL function to query the knowledge graph using vector search for the descriptions of the entities and the other entities and relationships they have in the graph. This data will also be fed into the prompt for context.

        
#### Response Generation without a knowledge graph:
    Using a template prompt in the RAG_prompts.py file a simple context window of the text data loaded earlier with the question is asked of the LLM model.

    

#### Response Generation with knowledge graph:
    Using a template prompt in the RAG_prompts.py file a context window of the text data loaded earlier, plus the graph data retrieved from SurrealDB with the question is asked of the LLM model.


##### notes:
    This notebook utilizes libraries :
        surrealdb to interact with SurrealDB
        google.generativeai to interact with Gemini
        networkx to visualize the graph

    The notebook is loosely based on a larger repository microsoft/graphrag. The microsoft/graphrag repository provides a more comprehensive, modular graph-based RAG system. This notebook focuses specifically on the knowledge graph construction aspect.

In [1]:
import sys
import os
import json
import google.generativeai as genai
import os
import time
import ipynb_path
from surrealdb import Surreal

from IPython.display import display, Markdown,JSON
from IPython.display import JSON


#get this notebook's path for access to the other files needed
dir_path = os.path.dirname(os.path.realpath(ipynb_path.get(__name__)))
sys.path.append(dir_path) #add the current directory for adding py imports
from RAG_prompts import STD_RAG_PROMT, GRAPH_RAG_PROMT

In [29]:

# this folder
nb_folder = dir_path
input_file = nb_folder + "/Operation Dulce v2 1 1.md"

out_folder = nb_folder + "/rag_{0}".format(time.strftime("%Y%m%d-%H%M%S"))
os.makedirs(out_folder, exist_ok=True)
chat_file = out_folder + "/chat.json"

messages_to_log = []



    
ip = "0.0.0.0:8080"
url = "ws://{0}/rpc".format(ip)

u = "root"
p = "root"
n = "graph_rag"
d = "graph_rag"
db_folder = nb_folder + "/db"

surrealdb_start = "surreal start --log none --allow-net --user {u} --pass {p} --bind {ip} \"rocksdb://{db_folder}\"".format(
    u=u,
    p=p,
    ip=ip,
    db_folder=db_folder)

#run this command if your surreal instance isn't running yet 
#copy and paste from below into a terminal
print(surrealdb_start)

#ensure the embedding service is running as well
"""
cd embedding_api
env FLASK_APP=embedding_api.py python -m flask --app embedding_api --debug run 
"""     

surreal start --log none --allow-net --user root --pass root --bind 0.0.0.0:8080 "rocksdb:///Users/sandro/Library/CloudStorage/GoogleDrive-apireno@gmail.com/My Drive/code/graph_rag/db"


'\ncd embedding_api\nenv FLASK_APP=embedding_api.py python -m flask --app embedding_api --debug run \n'

In [20]:

# get the text from the source file for use in our prompt
with open(input_file, "r") as f:
    input_text = f.read()


entity_types = ["PERSON", "PLACE"]
relation_type = "RELATED_TO"

#create the function that will return the portion of the knowledge graph 
#relevant to the question using vector search on the description embedding
retrieve_function_surql = """
DEFINE FUNCTION OVERWRITE fn::get_related_entities($text: string) {{
    
    LET $emb = fn::get_embeddings($text);
    
    RETURN SELECT id,description, ->{1}.{{out,strength,description}} FROM {0} 
        WHERE  embedding <|5,COSINE|> $emb;
}};
""".format(entity_types[0],relation_type)


async with Surreal(url) as db:
    await db.signin({"user": u, "pass": p}) 
    await db.use(n, d)
    outcome = await db.query(retrieve_function_surql)



In [21]:

#Let's test out vector/graph query
#we should see a collection of people and relationships
#where the person's description is related to the question 

async with Surreal(url) as db:
    await db.signin({"user": u, "pass": p}) 
    await db.use(n, d)
    get_the_graph_context_surql = "fn::get_related_entities('{0}')"
    outcome = await db.query(get_the_graph_context_surql.format("Who is the main character?"))

JSON(outcome)
    

<IPython.core.display.JSON object>

In [30]:


def generate_content_with_attachment(the_model,the_messages,attached_file):
    #time.sleep(5) # to stop generating too many requests per min

    response = the_model.generate_content([attached_file,str(the_messages)])
    return response





def generate_std_LLM_messages(the_question,attached_file):
    messages =  [
            {"role": "user", "parts": [{"text": STD_RAG_PROMT.format(
                question = the_question
            )}]}
        ]
    return messages


#this function will call the LLM with a prompt that includes the text of the original document
def get_std_LLM_response(the_model,the_question,attached_file):
    
    messages = generate_std_LLM_messages(the_question,attached_file)
    messages_to_log.append(messages)
    
    with open(chat_file, "w") as f:
        json.dump(messages_to_log, f)
        
    response = generate_content_with_attachment(model,messages,attached_file)
    return response

   
async def generate_graph_RAG_messages(the_db,the_question,attached_file):
    get_the_graph_context_surql = "fn::get_related_entities('{0}')"
    
    outcome = await the_db.query(get_the_graph_context_surql.format(the_question))
    messages = [
            {"role": "user", "parts": [{"text": GRAPH_RAG_PROMT.format(
                question = the_question,
                knowledge_graph = outcome
            )}]}
        ]
    
    return messages
    
    
#this function will call the LLM with a prompt that includes the text of the original document
#plus the relevant entities and relationships from the knowledge graph bases on a vector search
async def get_graph_RAG_response(the_db,the_model,the_question,attached_file):
    
    messages = await generate_graph_RAG_messages(the_db,the_question,attached_file)
    messages_to_log.append(messages)
    
    with open(chat_file, "w") as f:
        json.dump(messages_to_log, f)
    
    response = generate_content_with_attachment(model,messages,attached_file)
    return response
    
       

In [23]:

# Set your API key (I've stored mine in an env variable)
genai.configure(api_key=os.getenv('GOOGLE_GENAI_API_KEY'))

#flash is faster and good enough
#model = "gemini-1.0-pro"
model = "gemini-1.5-flash"

model = genai.GenerativeModel(model)


the_attached_file = genai.upload_file(input_file)

while the_attached_file.state.name == "PROCESSING":
    print(".", end="")
    time.sleep(10)
    the_attached_file = genai.get_file(the_attached_file.name)





In [47]:
q = "Who is the main character and what motivates them?"

print ("\n\n\n\n ------ STD RAG -------------------------\n\n\n\n")
messages = generate_std_LLM_messages(q,the_attached_file)
print(json.dumps(messages[0], indent=4, sort_keys=True))

print ("\n\n\n\n ------ GRAPH RAG -------------------------\n\n\n\n")

async with Surreal(url) as db:
    await db.signin({"user": u, "pass": p}) 
    await db.use(n, d)
    messages = await generate_graph_RAG_messages(db,q,the_attached_file)
    print(json.dumps(messages[0], indent=4, sort_keys=True))
 
   






 ------ STD RAG -------------------------




{
    "parts": [
        {
            "text": "\n-Goal-\nYou are a researcher who is providing answers to a question for you client based on documents provided you. Answer the questions to the best of your ability given the documents.\n\n<question>\nWho is the main character and what motivates them?\n</question>\n\n "
        }
    ],
    "role": "user"
}




 ------ GRAPH RAG -------------------------




{
    "parts": [
        {
            "text": "\n-Goal-\nYou are a researcher who is providing answers to a question for you client based on documents provided you. You also have access to a knowledge graph based on the documents provided. Answer the questions to the best of your ability given the documents and knowledge graph.\n\n<knowledge graph>\n[{'result': [{'->RELATED_TO': [{'description': 'Alex Mercer works at the central hub during the signal analysis.', 'out': 'PLACE:\u27e8CENTRAL HUB\u27e9', 'strength': 8}, {'description':

In [57]:
#Lets test a question...

q = "Who is the main character and what motivates them? Format your response in bullet points."

response = get_std_LLM_response(model,q,the_attached_file)

print ("\n\n\n\n ------ STD RAG -------------------------\n\n\n\n")



display(Markdown(response.text))

print ("\n\n\n\n ------ GRAPH RAG -------------------------\n\n\n\n")

async with Surreal(url) as db:
    await db.signin({"user": u, "pass": p}) 
    await db.use(n, d)
    response = await get_graph_RAG_response(db,model,q,the_attached_file)
    display(Markdown(response.text))








 ------ STD RAG -------------------------






* **Main Character:** While the story features a team, Agent Alex Mercer arguably functions as the protagonist.  He's central to many key scenes and interactions, and his internal struggles and growth are prominent throughout the narrative.

* **Motivations:**
    * **Duty and Compliance:** Mercer is initially motivated by a sense of duty and adherence to protocol, demonstrated by his initial compliant behavior during briefings.  This is a product of his military background and experience.
    * **Discovery and Understanding:**  As the story unfolds, a stronger motivation for discovery and understanding of the alien intelligence emerges. He demonstrates a growing curiosity and desire to grasp the full implications of their contact, moving beyond simple obedience.  This is seen in his interactions with Jordan Hayes, where they share an understanding of the deeper, more significant implications of the mission.
    * **Protection of Humanity:** Mercer's actions are increasingly driven by a desire to protect humanity from potential threats posed by the alien contact, balancing the pursuit of knowledge with the need for caution and strategic planning. This is shown in his leadership and decision-making throughout the operation.
    * **Overcoming Hierarchical Constraints:** A subtle but important motivation for Mercer is his ability to overcome the rigid hierarchical structures of Paranormal Military Squad. This is highlighted by his interactions with Taylor Cruz, where he progressively challenges and influences Cruz’s authoritarian approach. He finds common ground with other team members who share a desire for discovery that transcends protocols.







 ------ GRAPH RAG -------------------------






* **Main Character:** Alex Mercer is the main character.

* **Motivations:**
    * **Mission Success:** Alex's primary motivation is the successful completion of Operation: Dulce.  This involves deciphering the alien signals, understanding their intent, and ensuring the safety of his team.  His military background heavily influences this drive.
    * **Understanding the Unknown:**  Alex possesses a strong intellectual curiosity. He's driven by the desire to understand the nature of the alien intelligence and the implications of first contact. This desire transcends simple mission parameters; it’s a quest for knowledge.
    * **Team Leadership and Protection:** Alex demonstrates a strong sense of responsibility for his team. He acts as a leader, offering guidance, support, and protection while fostering trust and collaboration amongst his fellow agents.  His decisions are often tempered by his concern for the well-being of his team.
    * **Humanity's Future:**  Underlying Alex's motivations is a concern for humanity's future. He recognizes that their interaction with the alien intelligence could have profound and lasting consequences for the entire human race, influencing his approach to the mission.  He strives to ensure a positive outcome for humanity as a whole.



In [58]:

#Let's test a question that asks about the main character's relationships
q = "What places and people does Alex Mercer hold dearest? Format your response in bullet points. Format your response in bullet points."

response = get_std_LLM_response(model,q,input_text)

print ("\n\n\n\n ------ STD RAG -------------------------\n\n\n\n")


display(Markdown(response.text))

print ("\n\n\n\n ------ GRAPH RAG -------------------------\n\n\n\n")

async with Surreal(url) as db:
    await db.signin({"user": u, "pass": p}) 
    await db.use(n, d)
    response = await get_graph_RAG_response(db,model,q,input_text)
    display(Markdown(response.text))







 ------ STD RAG -------------------------






Based on the provided text, Alex Mercer's dearest places and people are not explicitly stated.  The narrative focuses on his professional relationships and the mission at hand.  However, we can infer some potential candidates:

* **Dr. Jordan Hayes:**  The text repeatedly highlights a strong professional bond and mutual respect between Alex and Jordan, suggesting a deep level of trust and understanding.  Their shared intellectual curiosity and willingness to push boundaries in the face of the unknown hints at a significant, possibly close, relationship.

* **Sam Rivera:** Alex shows a degree of mentorship and trust toward Sam, highlighting Sam's keen insights and recognizing their importance to the mission. This suggests a valued professional relationship, bordering on mentorship.


There is no mention of family, friends, or other personal relationships in the provided text.  The narrative heavily emphasizes his professional life and dedication to the mission.






 ------ GRAPH RAG -------------------------






Based on the provided text and knowledge graph, here's what we can infer about the places and people Alex Mercer values most:

* **People:**
    * **Jordan Hayes:** The knowledge graph indicates a strong bond between Alex and Jordan, highlighting their close collaboration, shared insights, Alex's trust and respect for Jordan's expertise (strength: 9).  The narrative emphasizes their shared understanding and reliance on each other.
    * **Sam Rivera:** The knowledge graph shows Alex mentoring Sam and trusting his technological expertise (strength: 8). The story also demonstrates Alex's belief in Sam's abilities and his willingness to rely on Sam's insights.

* **Places:**  The knowledge graph doesn't directly indicate places Alex *holds dearest*, but it highlights locations crucial to his mission and where he spends significant time:
    * **Dulce Base:** This is where the core of the operation takes place, and the knowledge graph shows Alex's strong connection to it (strength: 9).  His actions and focus within the base demonstrate his dedication to the mission and therefore, implicitly, his association with the place.
    * **Central Hub/Communications Hub/Mainframe Room:** These locations within Dulce Base are central to the signal analysis and communication with alien intelligence. Alex's frequent presence in these locations suggests their importance to his work and, by extension, his sense of purpose.


It's important to note that the knowledge graph primarily focuses on professional relationships and locations relevant to the mission.  The text hints at deeper sentiments but doesn't explicitly state which places or people Alex Mercer "holds dearest" in a personal sense.  The analysis focuses on professional relationships and places of critical importance to his mission.


In [78]:


q = """


Create a table of places that Alex communicates with:


In the first column indicate the person. 
In the 2nd column list the frequency he has scenes with them.
In the 3rd column list the places they had scenes together ordered by importance of the place and scene.
In the 4th column indicate the strength of the bond between the people.
In the 5th column score the total weight of the interactions across the scenes in aggregate.

output the response in a table format using markdown language.
"""


response = get_std_LLM_response(model,q,input_text)

print ("\n\n\n\n ------ STD RAG -------------------------\n\n\n\n")

display(Markdown(response.text))

print ("\n\n\n\n ------ GRAPH RAG -------------------------\n\n\n\n")

async with Surreal(url) as db:
    await db.signin({"user": u, "pass": p}) 
    await db.use(n, d)
    response = await get_graph_RAG_response(db,model,q,input_text)
    display(Markdown(response.text))





 ------ STD RAG -------------------------






## Alex Mercer's Communication Network

| Person             | Communication Frequency | Places of Communication (Ordered by Importance)                                                                   | Bond Strength | Interaction Weight Score |
|----------------------|--------------------------|-----------------------------------------------------------------------------------------------------------------|-----------------|--------------------------|
| Jordan Hayes         | Very High                 | 1. Dulce Base Command Center (multiple scenes, critical collaborations), 2. Dulce Base Corridors (shared concerns), 3. Elevator to Dulce (shared apprehension), 4. Dulce Base Lab (scientific collaboration) | Very Strong     | 9                        |
| Taylor Cruz          | High                      | 1. Dulce Base Command Center (conflicts and uneasy alliances), 2. Briefing Room (initial instructions, later collaboration), 3. Equipment Bay (philosophical disagreement) | Strong but strained | 7                        |
| Sam Rivera           | High                      | 1. Dulce Base Command Center (collaborative work, shared excitement), 2. Sam's Office (advice and encouragement), 3. Dulce Base Corridors (shared mission) | Strong           | 7                        |


**Note:**  The "Interaction Weight Score" is a subjective assessment based on the length and significance of their interactions throughout the provided text.  A higher score indicates more frequent and/or more impactful interactions.  The ordering of places within each person's row reflects the apparent importance of the setting to their relationship and the overall narrative.






 ------ GRAPH RAG -------------------------






Based on the provided text and knowledge graph, here's a table summarizing Alex Mercer's communication and interactions,  keeping in mind that the provided knowledge graph lacks information on the *frequency* of scenes and requires inference from the narrative.  The "Strength of Bond" and "Total Interaction Weight" columns are estimations based on the narrative and the knowledge graph's strength scores.  Precise quantification isn't possible without more structured data.

| Person           | Frequency of Scenes | Places of Interaction (ordered by importance)                                    | Strength of Bond | Total Interaction Weight (estimated) |
|-------------------|----------------------|---------------------------------------------------------------------------------|--------------------|------------------------------------|
| Jordan Hayes      | Very High             | Dulce Base Command Center, Mainframe Room, Communications Hub, Underground Meeting Room | 9                  | 90                                 |
| Sam Rivera        | High                  | Dulce Base Command Center, Mainframe Room, Server Room, Underground Meeting Room      | 8                  | 72                                 |
| Taylor Cruz       | High                  | Dulce Base Command Center, Mainframe Room, Underground Meeting Room, Briefing Room | 7                  | 63                                 |


**Notes:**

* **Frequency:**  This column is an approximation based on the narrative's emphasis on the interactions between characters.  "Very High" indicates frequent, significant interactions throughout the story. "High" suggests numerous important interactions.
* **Places:** The order of places reflects the narrative's emphasis on the importance of each location within the context of their interactions.  For instance, the Dulce Base Command Center is listed first because many major interactions occur there.
* **Strength of Bond:** This is a subjective assessment based on the descriptive language in the text and the strength scores from the knowledge graph.  It reflects the level of trust, cooperation, and understanding between characters.
* **Total Interaction Weight:** This is a rough estimate, multiplying the "Frequency" (interpreted as a numerical value: Very High=10, High=8) by the "Strength of Bond". It attempts to quantify the overall significance of the interactions.


The knowledge graph is limited. A more comprehensive graph that includes scene frequency and interaction details would allow for a more precise and detailed response.


In [73]:


q = """

Create a table of places that Alex visits:


In the first column indicate the place name. 
In the 2nd column indicate the frequency he visits them.
In the 3rd column list the people who were there with him by strength of the bond of the people.
In the 4th column indicate the importance of the place.
In the 5th column describe the location in 255 characters or less.
In the 6th column score the total weight of the interactions across the scenes in aggregate.

output the response in a table format using markdown language.
"""



response = get_std_LLM_response(model,q,input_text)

print ("\n\n\n\n ------ STD RAG -------------------------\n\n\n\n")

display(Markdown(response.text))

print ("\n\n\n\n ------ GRAPH RAG -------------------------\n\n\n\n")

async with Surreal(url) as db:
    await db.signin({"user": u, "pass": p}) 
    await db.use(n, d)
    response = await get_graph_RAG_response(db,model,q,input_text)
    display(Markdown(response.text))





 ------ STD RAG -------------------------






| Place Name             | Frequency | People Present (Bond Strength)                     | Importance                               | Location Description                                                                     | Interaction Weight |
|-------------------------|------------|----------------------------------------------------|-------------------------------------------|-----------------------------------------------------------------------------------------|--------------------|
| Dulce Base Command Center | Multiple    | Jordan Hayes (High), Taylor Cruz (Medium), Sam Rivera (Medium) | Extremely High                            | Sterile, high-tech underground command center in Dulce Base.                              | 8                 |
| Dulce Base Corridors     | Multiple    | Jordan Hayes (High), Taylor Cruz (Medium)          | High                                     | Long, dark, metallic corridors within Dulce Base.                                         | 6                 |
| Dulce Base Server Room   | Once        | Sam Rivera (High)                                 | Medium                                    | Cramped room with numerous servers.                                                        | 3                 |
| Briefing Room            | Multiple    | Jordan Hayes (Medium), Taylor Cruz (Medium), Sam Rivera (Medium) | High                                     | Sterile room for mission briefings.                                                      | 3                 |
| Dulce Base Equipment Bay | Once        | Taylor Cruz (High)                               | Low                                      | Secluded bay for checking equipment.                                                     | 2                 |
| Dulce Base Elevator Shaft | Once        | Jordan Hayes (High), Taylor Cruz (Medium), Sam Rivera (Medium) | Medium                                    | Underground shaft connecting levels of Dulce Base.                                       | 2                 |
| Deserted Corridor       | Once        | Taylor Cruz (High)                              | Low                                      | Long and empty corridor inside the Dulce facility                                      | 1                 |
| Dulce Base Mainframe Room| Once        | Jordan Hayes (High), Taylor Cruz (Medium), Sam Rivera (Medium) | High                                     | Central hub of Dulce Base containing the mainframe.                                       | 1                 |
| Dulce Base Central Hub   | Once        | Jordan Hayes (High), Taylor Cruz (Medium), Sam Rivera (Medium) | High                                     | Central area where the mainframe room is located.                                    | 1                 |
| Cramped Underground Meeting Room | Once      | Jordan Hayes (High), Taylor Cruz (Medium)          | Medium                                    | Small, underground room near the elevator.                                                  | 1                 |



**Bond Strength Key:**

* **High:** Strong, established trust and mutual respect.
* **Medium:** Working relationship, some level of trust.
* **Low:** Limited interaction, minimal established trust.


**Interaction Weight:**  A subjective score reflecting the significance and depth of Alex's interactions at each location across the story.  Higher scores indicate more crucial scenes and stronger relationships built.






 ------ GRAPH RAG -------------------------






| Place Name             | Frequency | People Present (by strength)                                      | Importance        | Location Description                                                                                                 | Total Interaction Weight |
|-------------------------|------------|--------------------------------------------------------------------|---------------------|-----------------------------------------------------------------------------------------------------------------|--------------------------|
| ⟨Dulce Base⟩           | Multiple   | Jordan Hayes (9), Sam Rivera (9), Taylor Cruz (9), Alex Mercer (10) | Extremely Important | The main setting for Operation Dulce, a large, secretive underground military base.                               | 37                       |
| ⟨Central Hub⟩         | Multiple   | Jordan Hayes (9), Sam Rivera (9), Alex Mercer (10), Taylor Cruz (8) | Very Important      | The central communication and analysis area of Dulce Base, used for decoding alien signals.                      | 36                       |
| ⟨Communications Hub⟩ | Multiple   | Jordan Hayes (9), Alex Mercer (8)                                 | Very Important      | A section of Dulce Base dedicated to analyzing and interpreting alien signals.                                     | 17                       |
| ⟨Mainframe Room⟩       | Multiple   | Jordan Hayes (9), Sam Rivera (9), Alex Mercer (8), Taylor Cruz (8) | Very Important      | The heart of Dulce Base's technological infrastructure, where the mainframe and servers are located.              | 34                       |
| ⟨Elevator Shaft⟩      | Multiple   | Jordan Hayes (8), Sam Rivera (8), Taylor Cruz (8), Alex Mercer (8) | Important           | The vertical shaft used to travel between levels of Dulce Base.                                                 | 32                       |
| ⟨Meeting Room⟩         | 2          | Jordan Hayes (7), Sam Rivera (7), Taylor Cruz (7), Alex Mercer (7) | Moderately Important | A standard briefing room where the team gathers before missions.                                               | 28                       |
| ⟨Underground Meeting Room⟩ | 2          | Jordan Hayes (7), Sam Rivera (7), Taylor Cruz (7), Alex Mercer (7) | Moderately Important | A secure underground briefing area in Dulce Base.                                                         | 28                       |
| CORRIDOR               | Multiple   | Jordan Hayes (7), Sam Rivera (7), Taylor Cruz (7), Alex Mercer (7) | Moderately Important | Generic corridors within Dulce Base, used for travel between various locations.                                 | 28                       |
| ⟨Server Room⟩          | 1          | Sam Rivera (9)                                                    | Important           | A room dedicated to the servers and data processing equipment.                                                   | 9                        |
| ⟨Crash Site⟩           | 1          | Jordan Hayes (8)                                                    | Moderately Important | An external location; the site of an extraterrestrial aircraft crash that provides samples for alien technology.    | 8                        |
| ⟨Equipment Bay⟩        | 1          | Taylor Cruz (8)                                                   | Moderately Important | A secured location within Dulce Base where equipment is stored and checked before missions.                       | 8                        |


**Note:**  Frequency is a qualitative assessment based on the narrative.  The "Total Interaction Weight" is the sum of the strength values from the knowledge graph for each location.  The table is sorted by this column in descending order.  The presence of individuals in a location is determined from the knowledge graph but the ordering is based on the strength of their relationships with Alex Mercer within the context of that location.  Locations with multiple visits are considered to have higher importance.


In [81]:



q = """
    Based on Alex Mercer strongest relationships and the places most important to the plot choose a
    scene that is the most emotionally charged and important to make the cover of the book. 
   Describe this cover in 20 words or less.
"""

response = get_std_LLM_response(model,q,input_text)

print ("\n\n\n\n ------ STD RAG -------------------------\n\n\n\n")

display(Markdown(response.text))

print ("\n\n\n\n ------ GRAPH RAG -------------------------\n\n\n\n")

async with Surreal(url) as db:
    await db.signin({"user": u, "pass": p}) 
    await db.use(n, d)
    response = await get_graph_RAG_response(db,model,q,input_text)
    display(Markdown(response.text))






 ------ STD RAG -------------------------






Dulce Base command center: Four figures bathed in alien light, deciphering a cosmic message.






 ------ GRAPH RAG -------------------------






Here's a book cover description based on your request:

**Image:** Alex Mercer and Jordan Hayes, illuminated by alien script on a Dulce Base console, faces etched with awe and apprehension.

**Title:** Operation: Dulce - First Contact


In [83]:


q = """Based on Alex Mercer strongest relationships and the places most important to the plot choose a
    scene that is the most emotionally charged and important to make the cover of the book. 
   Draw this cover in the syle of an 8-bit video game.
"""

response = get_std_LLM_response(model,q,input_text)

print ("\n\n\n\n ------ STD RAG -------------------------\n\n\n\n")

display(Markdown(response.text))

print ("\n\n\n\n ------ GRAPH RAG -------------------------\n\n\n\n")

async with Surreal(url) as db:
    await db.signin({"user": u, "pass": p}) 
    await db.use(n, d)
    response = await get_graph_RAG_response(db,model,q,input_text)
    display(Markdown(response.text))





 ------ STD RAG -------------------------






Based on the provided text, the most emotionally charged and important scene for the book cover, focusing on Alex Mercer's strongest relationships and key plot locations, would be the climax of the interspecies communication in Dulce Base's command center.  This scene encapsulates several crucial elements:

* **Strongest Relationships:** The scene involves Alex Mercer interacting directly with his team (Jordan, Taylor, and Sam), showcasing the trust and collaboration crucial to their success (and survival).  The scene highlights the bond formed through shared danger and the pursuit of knowledge.

* **Key Plot Locations:** Dulce Base itself is a central location, symbolizing mystery, danger, and the unknown. The command center within is where the most critical interactions occur.

**8-Bit Video Game Cover Design:**

Imagine an 8-bit style cover art.  The overall aesthetic should be reminiscent of classic NES or SNES games.


* **Background:** A stylized representation of Dulce Base's command center. Use simple geometric shapes to depict consoles, monitors (with glowing 8-bit alien glyphs streaming across them), and the overall feeling of a futuristic but slightly retro technological space.  A dark, shadowy background suggests the subterranean setting.  Consider a subtle flickering effect to represent the unstable nature of the base and the alien communications.

* **Foreground:** Alex Mercer, depicted in classic 8-bit style (think pixelated sprite), is centrally positioned.  He might be slightly larger than the other characters to emphasize his leadership role. He could be interacting with one of his team members, perhaps Jordan Hayes, who is also prominently displayed, representing their strong working relationship.  The other two team members (Taylor and Sam) could be slightly smaller in the background, surrounding the central figures, hinting at their collaborative effort.

* **Color Palette:** Primarily dark blues and purples to represent the underground setting and the mysterious alien technology.  Accents of bright green or teal could highlight the monitors and alien glyphs, emphasizing the alien communication.  Alex and Jordan could have slightly brighter colors than the rest to draw the eye.

* **Title:** "Operation: Dulce" in bold, pixelated 8-bit font, prominently displayed above or below the central image.

* **Overall Feel:** The image should convey a sense of urgency, mystery, and collaboration.  The 8-bit style should maintain a balance between retro charm and the high-stakes nature of the scene, hinting at the tension of first contact without explicitly showing violence or conflict.


In summary, the cover would be a visually compelling representation of the story's climax, using the 8-bit style to evoke a sense of nostalgia and adventure, while effectively conveying the emotional intensity and core themes of the book.






 ------ GRAPH RAG -------------------------






Here's a description of a book cover scene and its 8-bit representation, based on your provided text and knowledge graph.


**Scene Choice:** The most emotionally charged and important scene, considering Alex Mercer's strongest relationships (Jordan Hayes, Sam Rivera) and key locations (Dulce Base's central hub/mainframe room), is the moment when the team first establishes a two-way communication with the alien intelligence.  This is a culmination of their efforts and a major turning point in the narrative.  The central hub, a key location in the knowledge graph, is the perfect setting for this pivotal moment.


**8-bit Book Cover Description:**

The cover art would be rendered in classic 8-bit style, reminiscent of early NES or Game Boy games.

* **Background:** The background depicts the Dulce Base mainframe room.  Use simple, blocky shapes for the consoles, monitors (showing a stylized alien script scrolling across the screens), and walls. The color palette should be limited to a few shades of blue, green, and gray to create a dark, slightly ominous atmosphere.

* **Characters:**

    * **Alex Mercer:**  A larger sprite than the other characters, standing slightly back from the consoles.  He would be depicted with a determined expression, possibly a slight furrow in his brow, showing the weight of responsibility.  His sprite should use shades of brown and dark green.
    * **Jordan Hayes:**  A smaller sprite positioned near the main console, intensely focused on the screen.  Their sprite could be mostly light blue and purple.
    * **Sam Rivera:** Another smaller sprite, positioned near Alex but closer to the equipment.  Their sprite would use shades of yellow and orange to convey their youthful enthusiasm.
    * **Taylor Cruz:** A sprite slightly to the side, arms crossed, displaying a more stern and skeptical expression than the others. Colors would be in shades of red and grey.

* **Foreground:**  The foreground might include a few pixelated sparks or electrical effects around the consoles to heighten the sense of technological tension.

* **Text:** The title, "Operation: Dulce," should be in a bold, pixelated font, possibly in white or bright yellow against the dark background.  A subtitle, like "First Contact," could be placed underneath in a smaller font.


**Emotional Impact:** The 8-bit style, while simple, would effectively communicate the tension and importance of the scene.  The limited color palette and use of pixel art will evoke nostalgia for classic video games, creating a sense of both retro and futuristic appeal. The focus on Alex Mercer’s expression alongside his close colleagues will draw the reader into the story. The scene conveys the gravity of the moment: a critical point of contact between two civilizations.


This 8-bit cover would be instantly recognizable as a science fiction adventure and would intrigue readers interested in retro gaming aesthetics and high-stakes sci-fi stories.


In [ ]:


imagen = genai.ImageGenerationModel("imagen-3.0-generate-001")

result = imagen.generate_images(
    prompt="Fuzzy bunnies in my kitchen",
    number_of_images=4,
    safety_filter_level="block_only_high",
    person_generation="allow_adult",
    aspect_ratio="3:4",
    negative_prompt="Outside",
)

for image in result.images:
  print(image)

# Open and display the image using your local operating system.
for image in result.images:
  image._pil_image.show()





In [55]:

q = """Based on Alex Mercer strongest relationships and the places most important to the plot choose a
    scene that is the most emotionally charged and important to make the cover of the book. 
   Draw this cover in the syle of an 8-bit video game using imagen3.
"""

print ("\n\n\n\n ------ STD RAG -------------------------\n\n\n\n")
messages = generate_std_LLM_messages(q,the_attached_file)
print(messages[0]["parts"][0]["text"])

print ("\n\n\n\n ------ GRAPH RAG -------------------------\n\n\n\n")

async with Surreal(url) as db:
    await db.signin({"user": u, "pass": p}) 
    await db.use(n, d)
    messages = await generate_graph_RAG_messages(db,q,the_attached_file)
    print(messages[0]["parts"][0]["text"])








 ------ STD RAG -------------------------





-Goal-
You are a researcher who is providing answers to a question for you client based on documents provided you. Answer the questions to the best of your ability given the documents.

<question>
Based on Alex Mercer strongest relationships and the places most important to the plot choose a
    scene that is the most emotionally charged and important to make the cover of the book. 
   Draw this cover in the syle of an 8-bit video game using imagen3.

</question>

 




 ------ GRAPH RAG -------------------------





-Goal-
You are a researcher who is providing answers to a question for you client based on documents provided you. You also have access to a knowledge graph based on the documents provided. Answer the questions to the best of your ability given the documents and knowledge graph.

<knowledge graph>
[{'result': [{'->RELATED_TO': [{'description': 'Alex Mercer works at the central hub during the signal analysis.', 'out': 'PLA